# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data\weather_data.csv")
weather_data


,Number,City,Country,Latitude,Longitude,Temp,Humidity,Cloudiness,Windspeed
0,1,kruisfontein,ZA,-34.0033,24.7314,89.01,40,51,8.01
1,2,vila franca do campo,PT,37.7167,-25.4333,55.53,67,40,25.32
2,3,port alfred,ZA,-33.5906,26.8910,75.99,71,7,11.99
3,4,bredasdorp,ZA,-34.5322,20.0403,82.40,36,0,11.50
4,5,albany,US,42.6001,-73.9662,25.74,86,100,1.01
...,...,...,...,...,...,...,...,...,...
516,517,mehran,IR,33.1222,46.1646,62.60,27,0,4.61
517,518,gamboma,CG,-1.8764,15.8644,96.69,41,55,1.92
518,519,aneho,TG,6.2333,1.6000,91.40,58,20,10.36
519,520,eyl,SO,7.9803,49.8164,80.29,51,62,23.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Create a humidity Heatmap layer
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filter_weather_data = weather_data.loc[(weather_data.Temp < 80) & (weather_data.Temp > 70) & (weather_data.Windspeed < 10) & (weather_data.Cloudiness < 10)]

len(filter_weather_data)


18

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = filter_weather_data[["City", "Country", "Latitude", "Longitude"]]
hotel_df.insert(4, "Hotel Name", "") 
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
9,sao filipe,CV,14.8961,-24.4956,
39,busselton,AU,-33.6500,115.3333,
56,chuy,UY,-33.6971,-53.4616,
67,harindanga,IN,22.0383,88.3214,
87,riyadh,SA,24.6877,46.7219,
99,khanewal,PK,30.3000,71.9333,
153,susangerd,IR,31.5608,48.1831,
176,bankura,IN,23.2500,87.0667,
205,salalah,OM,17.0151,54.0924,
211,nouadhibou,MR,20.9310,-17.0347,


In [7]:
# Store parameters for url query
params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key
}

# Obtained from https://developers.google.com/places/web-service/search
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    latitude = row["Latitude"]
    longitude = row["Longitude"]

    params["location"] = f"{latitude},{longitude}"

    response = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Hotel not found"
        

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
9,sao filipe,CV,14.8961,-24.4956,La Fora Ecolodge
39,busselton,AU,-33.6500,115.3333,The Sebel Busselton
56,chuy,UY,-33.6971,-53.4616,Bertelli Chuí Hotel
67,harindanga,IN,22.0383,88.3214,Hotel not found
87,riyadh,SA,24.6877,46.7219,Four Seasons Hotel Riyadh At Kingdom Center
99,khanewal,PK,30.3000,71.9333,Tariq Kamboh Hotel
153,susangerd,IR,31.5608,48.1831,"Hall, Hotel And Restaurant Ghasr"
176,bankura,IN,23.2500,87.0667,OYO 47055 Mayaban Village
205,salalah,OM,17.0151,54.0924,HYATT SALALAH HOTEL
211,nouadhibou,MR,20.9310,-17.0347,Free Zone Hotel Nouadhibou


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))